In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch

import cv2
from os import path
from PIL import Image
import torch.utils.data as data
from torchvision import models, transforms
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import h5py
import timm

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from scipy.stats import pearsonr

In [ ]:
class args:
    seed = 22
    batch_size = 32
    epochs = 10
    image_size = 384  # scale shorter end of image to this size and centre crop
    central_fraction = 0.875  # only take this much of the centre when scaling and centre cropping
    load_img = False # whether load img or not
    workers = 2
    feature_extract = True

def seed_torch(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print("seed: ", seed)

seed_torch(args.seed)

In [ ]:
dir_path = '/kaggle/input/petfinder-pawpularity-score'
# dir_path = '../input/petfinder-pawpularity-score' # This is used in Kaggle Code

cached_dir = path.join(dir_path, 'cached_data')
train_img_dir = path.join(dir_path, 'train')
test_img_dir = path.join(dir_path, 'test')

df_train = pd.read_csv(path.join(dir_path, 'train.csv'))
df_test = pd.read_csv(path.join(dir_path, 'test.csv'))

train_img_paths = [path.join(train_img_dir, f"{img_id}.jpg") for img_id in df_train["Id"].values]
test_img_paths = [path.join(test_img_dir, f"{img_id}.jpg") for img_id in df_test["Id"].values]

In [ ]:
property_names = [col for col in df_train.columns if col not in ['Id','Pawpularity']]
property_names # 12 properties

# train metda data
train_meta_X = df_train[property_names]
train_Y = df_train['Pawpularity']

# test metda data
test_id = df_test['Id']
test_meta_X = df_test.drop('Id',axis=1)
test_meta_X.shape

In [ ]:
# Pytorch Data loader


class PawpularDataset(data.Dataset):
    def __init__(self, image_data, meta_features, labels, img_paths, augmentations=None):
        super(PawpularDataset, self).__init__()
        self.load_img = False
        self.image_data = image_data
        self.meta_features = meta_features
        self.labels = labels
        self.augmentations = augmentations
        self.image_paths = img_paths
        if self.augmentations is not None:
            self.load_img = True


    def __getitem__(self, item):
        if self.load_img:
            image = cv2.imread(self.image_paths[item])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            augmented = self.augmentations(image=image)["image"]
            image = np.transpose(augmented, (2, 0, 1)).astype(np.float32)
        else:
            image = self.image_data[item]

        meta = self.meta_features[item]
        label = self.labels[item]
        return image.astype('float32'), meta.astype('float32'), label

    def __len__(self):
        return len(self.labels)


def get_loader(split, features, meta_data, labels, img_paths=[], batch_size=args.batch_size, augmentations=None):
    """ Returns a data loader for the desired split """
    dataset = PawpularDataset(features, meta_data, labels, img_paths, augmentations)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True if split != 'test' else False,  # only shuffle the data in training
        pin_memory=True,
        num_workers=args.workers,
    )
    return loader

In [ ]:
import albumentations

def get_augmentations(train=True):
    if train:
        return albumentations.Compose(
            [
                albumentations.Resize(args.image_size, args.image_size, p=1),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.VerticalFlip(p=0.5),
                albumentations.Rotate(limit=180, p=0.7),
                albumentations.ShiftScaleRotate(
                    shift_limit = 0.1, scale_limit=0.1, rotate_limit=45, p=0.5
                ),
                albumentations.HueSaturationValue(
                    hue_shift_limit=0.2, sat_shift_limit=0.2,
                    val_shift_limit=0.2, p=0.5
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=(-0.1, 0.1),
                    contrast_limit=(-0.1, 0.1), p=0.5
                ),
                albumentations.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                    p=1.0,
                ),
            ]
        )
    else:
        return albumentations.Compose(
            [
                albumentations.Resize(args.image_size, args.image_size, p=1),
                albumentations.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                    p=1.0,
                ),
            ]
        )

augmentations = get_augmentations(train=True)
trainval_loader = get_loader('trainval', None, train_meta_X.values, train_Y.values.astype('float32'), train_img_paths,augmentations=augmentations)

augmentations = get_augmentations(train=False)
test_loader = get_loader('test', None, test_meta_X.values, torch.zeros(len(test_img_paths)), test_img_paths, augmentations=augmentations)

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class PretrainedCNN(nn.Module):
    def __init__(self, model_name='resnet', use_meta=False):
        super(PretrainedCNN, self).__init__()
        if model_name == 'resnet':
            self.model = models.resnet152(pretrained=True)
            set_parameter_requires_grad(self.model, feature_extracting=True)
            num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 128)
        self.drop = nn.Dropout(0.5)
        self.use_meta =use_meta
        if self.use_meta:
            self.fc = nn.Linear(128+12, 1)
        else:
            self.fc = nn.Linear(128, 1)
        

    def forward(self, img_data, meta=None, targets=None):
        x = self.model(img_data)
        if self.use_meta:
            x = torch.cat([x, meta], dim=1)
        x = self.fc(self.drop(x)) # [b, o]
        return x



class BaseCNN(nn.Module):
    def __init__(self, num_filters=[], use_meta=False):
        super(BaseCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, num_filters[0], kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters[0], num_filters[1], kernel_size = 4, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(num_filters[1], num_filters[2], kernel_size = 5, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters[2] ,num_filters[3], kernel_size = 8, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(num_filters[3], num_filters[4], kernel_size = 6, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(num_filters[4], num_filters[4], kernel_size = 6, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(12544, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,128)
        )
        # fc
        self.use_meta = use_meta
        self.drop = nn.Dropout(0.5)
        if self.use_meta:
            self.fc = nn.Linear(128+12, 1)
        else:
            self.fc = nn.Linear(128, 1)


    def forward(self, img_data, meta=None, targets=None):
        o = self.conv(self.drop(img_data)) # [b, 1, m]
        if self.use_meta:
            o = torch.cat([o, meta], dim=1)
        x = self.fc(self.drop(o)) # [b, o]
        return x


class BaseMLP(nn.Module):
    def __init__(self, use_meta=False, input_size=args.image_size):
        super(BaseMLP, self).__init__()
        # fc
        self.use_meta = use_meta
        self.drop = nn.Dropout(0.5)
        self.lin = nn.Sequential(
            nn.Linear(input_size*input_size, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(3),
            nn.Dropout(0.5),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(3),
            nn.Dropout(0.5),
            nn.Linear(512,128)
        )
        if self.use_meta:
            self.fc = nn.Linear((128*3)+12, 1)
        else:
            self.fc = nn.Linear(128*3, 1)


    def forward(self, img_data, meta=None, targets=None):
        b, c, img_size, _ = img_data.shape
        img_data = img_data.view(b, c, img_size*img_size)
        o = self.lin(self.drop(img_data)) # [b, 3, 128]
        o = o.view(b, -1)
        if self.use_meta:
            o = torch.cat([o, meta], dim=1)
        x = self.fc(self.drop(o)) # [b, o]
        return x


class PretrainedTransformer(nn.Module):
    def __init__(self, model_name='', use_meta=False, pretrained=True):
        super(PretrainedTransformer, self).__init__()
        # fc
        self.use_meta = use_meta
        self.drop = nn.Dropout(0.5)

        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=0, in_chans=3)
        set_parameter_requires_grad(self.model, feature_extracting=True)
        num_features = self.model.num_features

        if self.use_meta:
            self.fc = nn.Linear(num_features+12, 1)
        else:
            self.fc = nn.Linear(num_features, 1)


    def forward(self, img_data, meta=None, targets=None):
        x = self.model(img_data)
        if self.use_meta:
            x = torch.cat([x, meta], dim=1)
        x = self.fc(self.drop(x)) # [b, o]
        return x




torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [ ]:
# train 
def train(model, optimizer, train_loader, val_loader, train=True, val=False, epoch=0, grad_clip=False):
    labels, preds = [], []
    if train:
        mse = nn.MSELoss()
        train_loader = tqdm(train_loader, desc='{} E{:03d}'.format('train', epoch), ncols=0)
        model.train()
        for i, (img, meta_feature, label) in enumerate(train_loader):
            img = img.cuda().float()
            meta_feature = meta_feature.cuda()
            label = label.cuda()
            pred = model(img, meta_feature, label)
            loss = mse(pred, label.view(-1, 1))
            loss.backward()
            if grad_clip:
                nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            optimizer.zero_grad()
    
    if val:
        model.eval()
        val_loader = tqdm(val_loader, desc='{} E{:03d}'.format('val', epoch), ncols=0)
        for i, (img, meta_feature, label) in enumerate(val_loader):
            img = img.cuda().float()
            pred = model(img, meta_feature, None)

            labels.append(label.detach().cpu())
            preds.append(pred.detach().cpu())

        labels = torch.cat(labels, dim=0).numpy() # [num_seg]
        preds = torch.cat(preds, dim=0).numpy() # [num_seg]

    return labels, preds 

In [ ]:
import torch.optim as optim

name = 'tf_efficientnet_b0_ns'
model = PretrainedTransformer(name, use_meta=False, pretrained=False)

params_to_update = model.parameters()
print("Params to learn:")
if args.feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
            
optimizer = optim.Adam(params_to_update, lr=1e-2, amsgrad=False)
model = nn.DataParallel(model).cuda()

In [ ]:
# # # Train & val the model
# best_score = 0
# epochs = args.epochs
# for epoch in range(epochs):
#     labels, pred = train(model, optimizer, trainval_loader, None, train=True, val=False, epoch=epoch, grad_clip=True)
#     torch.save({"model": model.state_dict()}, "efficient_model.pth")

In [ ]:
# Test the model
logs = torch.load("../input/efficient-model-paw/efficient_model.pth")
model.load_state_dict(logs['model'])
_, pred = train(model, optimizer, None, test_loader, train=False, val=True, epoch=0, grad_clip=False)
print("score:{}".format(pred.mean().item()))

submission = pd.DataFrame({'Id':test_id,'Pawpularity':pred.squeeze(1)})

In [ ]:
submission.to_csv("submission.csv", index=False)